<a href="https://colab.research.google.com/github/Mjh9122/ML_lit_review/blob/main/word2vec/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Efficient Estimation of Word Representations in Vector Space
## Authors: Tomas Mikolov, Kai Chen, Greg Corrado, Jeffrey Dean
### Notes: Michael Holtz

### Abstract

They describe two NN architectures for embedding words into vector spaces. The quality is measured via a "word similarity task," and they find higher accuracy at much lower computational cost. Furthermore, the vectors produced provide state-of-the-art performance on a test set for syntactic and semantic word similarities.

### Intro

Many previous NLP systems treat words as simply an element in a set of words, with no notion of similarity between words. These simple techniques have notable limits in many tasks. While trillions of words might be necessary to achieve performance with these simple methods, tasks such as automatic speech recognition or machine translation may have corpora with only millions or billions of words. In these scenarios, a more complex strategy is needed.

### Paper Goals

The goal is to create high-quality vector embeddings for millions of words from a billion+ word corpora. The expectation of the embedding is that similar words should be close to one another and that words can have multiple degrees of similarity (such as a similar ending). More surprising is that algebraic operations on these vectors hold their meaning. Ex. King - man + woman = queen. They also develop a test set for syntactic and semantic regularities and discuss how time and accuracy depend on embedding dimension.


### Previous work

Previous attempts at word embeddings via a neural network language model. The first proposed models learned both a word vector representation and a statistical language model. Later architectures attempted to learn the embedding via a single hidden layer and then the vectors were used to train the NNLM. Other work also found that NLP tasks became easier when working with word vectors. The architecture in this paper seeks to find these vectors in a much more computationally efficient way.

### NNLM Architectures

#### Feedforward NNLM
This model takes in N-words encoded in one-of-V coding. The input layer is then projected to a projection layer P. This layer is passed to a hidden layer, which in turn predicts a probability distribution over the 1xV output layer.

#### Recurrent NNLM (RNNLM)
This model removes the need to specify the context length for the input. The RNNLM removes the projection layer, consisting of only input, hidden, and output layers. It is a recurrent architecture becuase there are time delayed connections from the hidden layer to itself. These connection theoretically allow for short term memory, allowing past words to influence future predictions.


### New log-linear models
The main focus of the paper. New models are proposed which avoid the nonlinear nature of the neural nets above, allowing for much more efficient training. These new models can then be used to train the above architectures on a much smaller input dimension.

#### CBOW
Continous bag of words (CBOW) is similar to the feedforward model but there is no non-linear hidden layer. Instead the projection layer is shared for all words, and input contains words from the past as well as the future. The goal is to classify the word in the middle of the input.

In [2]:
# @title Imports
import torch
import torch.nn as nn

import random
import re

from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset

In [3]:
# @title Corpus

with open('the_cat_in_the_hat.txt', 'r') as f:
    corpus = f.read()

corpus_filtered = re.sub('[^A-Za-z0-9 ]+', " ", corpus)
corpus = corpus_filtered.lower().split()

In [4]:
# Create vocab
vocab = set(corpus)
vocab_size = len(vocab)
print(f'vocab size: {vocab_size}')

# To convert from word to embedding index
word_to_ix = {word:ix for ix, word in enumerate(vocab)}
ix_to_word = {ix:word for ix, word in enumerate(vocab)}

# Create dataset by taking four words on either side of a target word
context_length = 4

Xs = []
ys = []
for i in range(context_length, len(corpus) - context_length):
    context = corpus[i - context_length: i] + corpus[i + 1 : i + context_length + 1]
    target = corpus[i]
    Xs.append(context)
    ys.append(target)

Xs_t = [torch.tensor([word_to_ix[w] for w in x]) for x in Xs]
ys_t = [torch.tensor([word_to_ix[y]]) for y in ys]

vocab size: 242


In [5]:
# Quick dataset class for dataloading
class CBOW_Dataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.Y[idx],self.X[idx]

In [6]:
# CBOW model proper. No non-linear activations like the paper says.
class CBOW(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear = nn.Linear(embedding_dim, vocab_size)
        self.softmax = nn.LogSoftmax(dim = -1)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).sum(dim=1)
        out = self.linear(embeds)
        return self.softmax(out)

    def get_word_emdedding(self, word):
        word = torch.tensor(word_to_ix[word])
        return self.embeddings(word).view(1,-1)

In [7]:
# Create model instance, loss, optimizer, dataset, and dataloader
model = CBOW(vocab_size, 64)
loss_function = nn.NLLLoss()
optim = torch.optim.SGD(model.parameters(), lr=0.1)

dataset = CBOW_Dataset(Xs_t, ys_t)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=256, shuffle=True)

# training loop
for epoch in range(1000):
    epoch_loss = 0
    iters = 0
    correct = 0

    for labels, features in dataloader:
        optim.zero_grad()
        y_pred = model(features)
        correct += sum(torch.argmax(y_pred, dim=1) == labels.squeeze())
        loss = loss_function(y_pred, labels.squeeze())
        loss.backward()
        optim.step()
        epoch_loss += loss
        iters += 1

    if not epoch % 50:
        print(f'Epoch: {epoch}, Loss {epoch_loss.item()/iters}, Accuracy: {correct/len(Xs_t)}, Incorrect classifications: {(len(Xs_t) - correct)}/{len(Xs_t)}')


Epoch: 0, Loss 6.701642172677176, Accuracy: 0.008604793809354305, Incorrect classifications: 1613/1627
Epoch: 50, Loss 1.812006950378418, Accuracy: 0.5593116283416748, Incorrect classifications: 717/1627
Epoch: 100, Loss 1.2536566598074776, Accuracy: 0.7264904975891113, Incorrect classifications: 445/1627
Epoch: 150, Loss 0.9464400155203683, Accuracy: 0.8063921332359314, Incorrect classifications: 315/1627
Epoch: 200, Loss 0.7584351812090192, Accuracy: 0.8709281086921692, Incorrect classifications: 210/1627
Epoch: 250, Loss 0.6023101806640625, Accuracy: 0.8973571062088013, Incorrect classifications: 167/1627
Epoch: 300, Loss 0.5149840627397809, Accuracy: 0.925015389919281, Incorrect classifications: 122/1627
Epoch: 350, Loss 0.40927726881844656, Accuracy: 0.9446834921836853, Incorrect classifications: 90/1627
Epoch: 400, Loss 0.35207765442984446, Accuracy: 0.9594345688819885, Incorrect classifications: 66/1627
Epoch: 450, Loss 0.28885459899902344, Accuracy: 0.9686539769172668, Incorrec

In [ ]:
# Test the model on a sample sentance from the text
test = 'and then something went bump how that made us'
test = test.lower().split()
context = test[:4] + test[5:]
target = test[4]

test_vec = torch.tensor([word_to_ix[w] for w in context]).unsqueeze(0)
log_probs = model(test_vec)
pred = ix_to_word[torch.argmax(log_probs).item()]

print(f'Test context: {context}')
print(f'Test target: {target}')
print(f'Test prediction: {pred}')

Test context: ['and', 'then', 'something', 'went', 'how', 'that', 'made', 'us']
Test target: bump
Test prediction: bump
